In [1]:
import os
import glob
import shutil
from pathlib import Path
import numpy
import tensorflow as tf

from model_builder import model_builder, relabel, class_merger, balancer
import tools_keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet_v2, vgg19, efficientnet

In [6]:
specs = {
    'chips': "../chips/32/",
    'folder': "../urbangrammar_samba/spatial_signatures/ai/nw_32/",
}

In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        specs['chips'] + 'train',
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
validation_generator = validation_datagen.flow_from_directory(
        specs['chips'] + 'validation',
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

secret_datagen = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)
secret_generator = secret_datagen.flow_from_directory(
        specs['chips'] + 'secret',
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse')

Found 107162 images belonging to 14 classes.
Found 35721 images belonging to 14 classes.
Found 35729 images belonging to 14 classes.


In [4]:
model_specs = {
    'meta_class_map': {},
    'meta_class_names': [],
    'meta_chip_size': 32,
}


## using all classes

In [9]:
# initialise base model
base = keras.applications.VGG19(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False,
)
base.trainable = False
inputs = keras.Input(shape=(224, 224, 3))
x = vgg19.preprocess_input(inputs)
x = base(x, training=False)
# add bridge
x = layers.GlobalAveragePooling2D()(x)
 # add Dense relu layer
x = layers.Dense(512, activation="relu")(x)
# add softmax classfier
predictions = layers.Dense(14, activation="softmax")(x)

model = keras.Model(
        inputs,
        predictions,
        name=f"vgg19_pooling_512_14"
)
model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

In [10]:
# this will break after fitting with the new data input
h = tools_keras.fit_phase(
        model,
        train_generator,
        validation_generator,
        secret_generator,
        log_folder=specs["folder"] + "logs",
        pred_folder=specs["folder"] + "pred",
        model_folder=specs["folder"] + "model",
        json_folder=specs["folder"] + "json",
        specs=model_specs,
        epochs=250,
        patience=5,
        verbose=True,

    )

Model: "vgg19_pooling_512_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_2 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 14)       

2022-01-20 15:06:49.889839: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-20 15:06:49.889910: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-20 15:06:50.052687: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-20 15:06:50.052815: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed


Epoch 1/250
   1/3349 [..............................] - ETA: 33:13 - loss: 2.5238 - accuracy: 0.2188

2022-01-20 15:06:50.822722: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-20 15:06:50.822766: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


   2/3349 [..............................] - ETA: 18:04 - loss: 2.6300 - accuracy: 0.2188

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3349/3349 [==============================] - 449s 134ms/step - loss: 0.8303 - accuracy: 0.6861 - val_loss: 1.1044 - val_accuracy: 0.6140
Epoch 40/250
3349/3349 [==============================] - 449s 134ms/step - loss: 0.8276 - accuracy: 0.6889 - val_loss: 1.0985 - val_accuracy: 0.6128
Epoch 41/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.8250 - accuracy: 0.6883 - val_loss: 1.1199 - val_accuracy: 0.6107
Epoch 42/250
3349/3349 [==============================] - 449s 134ms/step - loss: 0.8234 - accuracy: 0.6902 - val_loss: 1.1076 - val_accuracy: 0.6141
Epoch 43/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.8183 - accuracy: 0.6922 - val_loss: 1.1384 - val_accuracy: 0.6173
Epoch 44/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.8165 - accuracy: 0.6914 - val_loss: 1.1176 - val_accuracy: 0.6183
Epoch 45/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.8110 - accuracy: 0.6936 - val

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



3349/3349 [==============================] - 449s 134ms/step - loss: 0.7840 - accuracy: 0.7047 - val_loss: 1.1515 - val_accuracy: 0.6158
Epoch 56/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7835 - accuracy: 0.7045 - val_loss: 1.1503 - val_accuracy: 0.6136
Epoch 57/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7765 - accuracy: 0.7060 - val_loss: 1.1829 - val_accuracy: 0.6108
Epoch 58/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7768 - accuracy: 0.7063 - val_loss: 1.1473 - val_accuracy: 0.6192
Epoch 59/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7750 - accuracy: 0.7061 - val_loss: 1.1600 - val_accuracy: 0.6109
Epoch 60/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7733 - accuracy: 0.7082 - val_loss: 1.1704 - val_accuracy: 0.6109
Epoch 61/250
3349/3349 [==============================] - 448s 134ms/step - loss: 0.7690 - accuracy: 0.7095 - val

AttributeError: 'DirectoryIterator' object has no attribute 'concatenate'

## using merged classes

In [8]:
from itertools import chain
import numpy as np

# combine classes
group_mapping = {
    0: ['9_0', '9_1', '9_2', '9_4', '9_5', '2_0', '2_1', '2_2'],
    1: ['1_0', '3_0', '5_0', '6_0', '8_0'],
    2: ['0_0', '4_0', '7_0']
}
all_groups = sorted(chain(*group_mapping.values()))
# define a mapping from old classes to new classes (i.e. 0,1 -> 0 and 2,3 -> 1)
# the original classes are sorted and turned to ints
old_to_new = []
for g in all_groups:
    for k in group_mapping.keys():
        if g in group_mapping[k]:
            old_to_new.append(k)
old_to_new = np.array(old_to_new).astype(int)

# the wrapping generator
def merged_classes(generator):
    for data, labels in generator:
        labels = old_to_new[labels.astype(int)]

        yield data, labels

        
# initialise base model
base = keras.applications.VGG19(
    weights="imagenet",
    input_shape=(224, 224, 3),
    include_top=False,
)
base.trainable = False
inputs = keras.Input(shape=(224, 224, 3))
x = vgg19.preprocess_input(inputs)
x = base(x, training=False)
# add bridge
x = layers.GlobalAveragePooling2D()(x)
 # add Dense relu layer
x = layers.Dense(512, activation="relu")(x)
# add softmax classfier
predictions = layers.Dense(3, activation="softmax")(x)

model = keras.Model(
        inputs,
        predictions,
        name=f"vgg19extended_pooling_512_3"
)
model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=["accuracy"],
    )

# this will break after fitting with the new data input
h = tools_keras.fit_phase(
        model,
        merged_classes(train_generator),
        merged_classes(validation_generator),
        merged_classes(secret_generator),
        log_folder=specs["folder"] + "logs",
        pred_folder=specs["folder"] + "pred",
        model_folder=specs["folder"] + "model",
        json_folder=specs["folder"] + "json",
        specs=model_specs,
        epochs=250,
        patience=5,
        verbose=True,
#         steps_per_epoch=107162 // 32  # total train // batch size
    )

Model: "vgg19extended_pooling_512_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_1 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_1 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 3) 

2022-01-20 14:02:34.573630: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-20 14:02:34.573656: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-20 14:02:34.702167: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-20 14:02:34.702328: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed


Epoch 1/250
      1/Unknown - 1s 520ms/step - loss: 1.4285 - accuracy: 0.2188

2022-01-20 14:02:35.373133: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-20 14:02:35.373164: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


      2/Unknown - 1s 289ms/step - loss: 1.9079 - accuracy: 0.3125

2022-01-20 14:02:35.676576: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-01-20 14:02:35.677019: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed
2022-01-20 14:02:35.692316: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 154 callback api events and 151 activity events. 
2022-01-20 14:02:35.694734: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-20 14:02:35.725759: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35

2022-01-20 14:02:35.756118: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14

      4/Unknown - 1s 246ms/step - loss: 1.6831 - accuracy: 0.4297

2022-01-20 14:02:35.884515: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35
Dumped tool data for xplane.pb to ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35/85c0885614ed.xplane.pb
Dumped tool data for overview_page.pb to ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35/85c0885614ed.overview_page.pb
Dumped tool data for input_pipeline.pb to ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35/85c0885614ed.input_pipeline.pb
Dumped tool data for tensorflow_stats.pb to ../urbangrammar_samba/spatial_signatures/ai/nw_32/logs/vgg19extended_pooling_512_3/train/plugins/profile/2022_01_20_14_02_35/85c0885614ed.tensorf

  37870/Unknown - 3840s 101ms/step - loss: 0.4134 - accuracy: 0.8268

KeyboardInterrupt: 